## HW1
1. Inpute live video from webcam.
2. Generate and update <b>Adaptive Background Image</b> using temporal alpha blending, i.e.,  <i>B(t)=alpha*B(t-1)+(1-alpha)*I(t)</i>.
3. Perform <b>Adaptive Background Subtraction</b>, i.e., <i>S(t)=abs(I(t)-B(t))</i>.
4. Perform <b>Foreground Detection</b>. i.e., <i>F(t)=I(t), if S(t) > Threshold</i>
5. Synthesis an image by keeping the foreground, and replacing the background using any virtual scene image of your choice.
6. Show the captured image <i>I(t)</i>, the adaptive background image <i>B(t)</i>, the subtracted image <i>S(t)</i>, the foreground image <i>F(t)</i>, and the virtual systhesis image.
6. Adjust the <i>alpha</i> value and observe the result.
7. Any idea on how to generate a better Adaptive Background Image? Try it and compare the results.
8. Write down your observation and comparison at the end of your code by adding an individual cell in Jupyter.
9. Upload your Jupyter code file (*.ipynb)

In [8]:
import cv2
import numpy as np

In [17]:
# To function help us to list available cameras on PC

def list_available_cameras(max_index):
    available_cameras = []
    for i in range(max_index):
        cap = cv2.VideoCapture(i)
        
        # If camera can be open
        if cap.isOpened():
            available_cameras.append(i) # Append to the array
            cap.release()               # Release camera then handle next
    return available_cameras

max_camera_index = 10
available_cameras = list_available_cameras(max_camera_index)

print("Available Cameras:", available_cameras)

Available Cameras: [0, 2]


[ WARN:0@2662.746] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@2662.748] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@2662.751] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@2662.752] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@2662.752] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@2662.753] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@2662.753] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@2662.754] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range
[ WARN:0@2662.754] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@2662.754] global obsensor_uvc_stream_channel.c

In [18]:

threshold = 10

def foreground_detection(frame, bt_frame, threshold):
    
    # Apply Averaging blur to smooth both the current frame and the background frame
    frame_blurred = cv2.blur(frame, (21, 21), 0)
    bt_frame_blurred = cv2.blur(bt_frame, (21, 21), 0)
    
    diff = cv2.absdiff(frame_blurred, bt_frame_blurred)
    
    fg_mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    _, fg_mask = cv2.threshold(fg_mask, threshold, 255, cv2.THRESH_BINARY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    fg_mask = cv2.erode(fg_mask,kernel, iterations=1)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)
    
    return fg_mask

def calculate_motion(frame, background):
    # Dummy function to calculate the amount of motion
    # This should be replaced with actual motion calculation
    motion_score = np.sum(cv2.absdiff(frame, background)) / frame.size
    return motion_score

cap = cv2.VideoCapture(2)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

ret, t_frame = cap.read()
bt_frame = cv2.resize(t_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

background_image = cv2.imread('Picture.jpg')

while True:

    ret, t_frame = cap.read()
    if not ret:
        break  
    t_frame_resized = cv2.resize(t_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    motion_score = calculate_motion(t_frame_resized, bt_frame)
    alpha = min(max(0.01, 1 - motion_score), 0.5)

    t_frame = cv2.resize(t_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    
    st_frame = cv2.absdiff(t_frame, bt_frame)
    t_frame = cv2.addWeighted(t_frame_resized, alpha, bt_frame, 1 - alpha, 0)
    mask = foreground_detection(t_frame,bt_frame, threshold)

    ft_frame = t_frame.copy()
    ft_frame[mask != 0] = t_frame[mask != 0]
    ft_frame[mask == 0] = [0, 255, 0]

    rows, cols = ft_frame.shape[:2]
    background_image = cv2.resize(background_image, (cols, rows))
    
    background_frame = background_image.copy()
    background_frame[mask != 0] = t_frame[mask != 0]
    
    #display the image
    cv2.imshow('Original Frame', t_frame)
    cv2.imshow('Adaptive Background Image', bt_frame)
    cv2.imshow('Adaptive Background Subtraction', st_frame)
    cv2.imshow('Foreground Mask', ft_frame)
    cv2.imshow('Back Image with Foreground', background_frame)

    if cv2.waitKey(1) &0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [6]:
alpha = 0.5
threshold = 10

def foreground_detection(frame, bt_frame, threshold):
    
    # Apply Averaging blur to smooth both the current frame and the background frame
    frame_blurred = cv2.blur(frame, (21, 21), 0)
    bt_frame_blurred = cv2.blur(bt_frame, (21, 21), 0)
    
    diff = cv2.absdiff(frame_blurred, bt_frame_blurred)
    
    fg_mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    _, fg_mask = cv2.threshold(fg_mask, threshold, 255, cv2.THRESH_BINARY)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    
    fg_mask = cv2.erode(fg_mask,kernel, iterations=1)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)
    
    return fg_mask

cap = cv2.VideoCapture(3)

if not cap.isOpened():
    raise IOError("Cannot open webcam")

ret, t_frame = cap.read()
bt_frame = cv2.resize(t_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

background_image = cv2.imread('Picture.jpg')

while True:

    ret, t_frame = cap.read()

    t_frame = cv2.resize(t_frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    
    st_frame = cv2.absdiff(t_frame, bt_frame)
    bt_frame = cv2.addWeighted(t_frame, alpha, bt_frame, 1 - alpha, 0)
    mask = foreground_detection(t_frame,bt_frame, threshold)

    ft_frame = t_frame.copy()
    ft_frame[mask != 0] = t_frame[mask != 0]
    ft_frame[mask == 0] = [0, 255, 0]

    rows, cols = ft_frame.shape[:2]
    background_image = cv2.resize(background_image, (cols, rows))
    
    background_frame = background_image.copy()
    background_frame[mask != 0] = t_frame[mask != 0]
    
    #display the image
    cv2.imshow('Original Frame', t_frame)
    cv2.imshow('Adaptive Background Image', bt_frame)
    cv2.imshow('Adaptive Background Subtraction', st_frame)
    cv2.imshow('Foreground Mask', ft_frame)
    cv2.imshow('Back Image with Foreground', background_frame)

    if cv2.waitKey(1) &0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Experiment pictures

### Captured image I(t)
<img src="Original Frame.png" alt="Captured image I(t)" style="float: left; margin-right: 10px;" />
<br style="clear: both;"/>

### Adaptive background image B(t)
<img src="Adaptive Background Image.png" alt="Adaptive background image B(t)" style="float: left; margin-right: 10px;" />
<br style="clear: both;"/>

### Adaptive Background subracted S(t)
<img src="Adaptive Background_subracted.png" alt="Adaptive background image B(t)" style="float: left; margin-right: 10px;" />
<br style="clear: both;"/>

### Foreground image F(t)
<img src="Foreground Mask.png" alt="Adaptive background image B(t)" style="float: left; margin-right: 10px;" />
<br style="clear: both;"/>

### Virtual systhesis image
<img src="Virtual systhesis image.png" alt="Adaptive background image B(t)" style="float: left; margin-right: 10px;" />
<br style="clear: both;"/>



## Experiment of alpha value

### alpha value in the role
The alpha value controls the rate at which the Adaptive Background model is updated, because it determines how the model updates over time to adapt to changes in the scene.

### When alpha value is higher
The model is more sensitive and can quickly adapt to changes in the scene, interestingly when the moving objects that are stationary for a short period of time are quickly counted as part of the background.

### When alpha value is lower
The model is more slower to respond to new information and more inclined to maintain the status, not susceptible to frame-by-frame changes, The advantage of helping when the background is relatively stable, or needs to resist interference caused by camera shake, light changes, etc, on the other hand, model updates too slowly to reflect these changes in a timely manner

## How to generate a better Adaptive Background Image?
We have many ways to improve our result, I use Dynamic Adjustment of 'alpha' to solve the problem, Instead of using a fixed alpha value for all frames, dynamically adjust alpha based on the level of motion detected in the scene. In scenes with minimal motion, smaller alpha can be used to adapt to background changes more quickly. The opposite solution, using larger alpha can maintain a stable background model.

The result